In [252]:
import random
import json
from pprint import pprint
import nltk
import re
import string
from nltk.tag.stanford import StanfordPOSTagger
from nltk.tag.stanford import StanfordPOSTagger
post = StanfordPOSTagger('lib/stanford-postagger-2014-08-27/models/english-bidirectional-distsim.tagger', 'lib/stanford-postagger-2014-08-27/stanford-postagger.jar', 'utf-8') # doctest: +SKIP

### make a random sample of 2 and smaple of 5 from the json data as starting data.
Note: only run this code once. Currently commented out. 

In [10]:
# with open('movies-with-roles-summaries.json')as data_file:
#     data = json.load(data_file)

# print(type(data))

<class 'dict'>


In [21]:
# data_sample_2 = random.sample((data.items()), 2)

In [16]:
# data_sample_5 = random.sample((data.items()), 5)

In [22]:
# with open('data_sample_2.json', 'w') as outfile:
#     json.dump(data_sample_2, outfile)

In [26]:
# with open('data_sample_5.json', 'w') as outfile:
#     json.dump(data_sample_5, outfile)

###Load in the sample of five

In [98]:
with open('data_sample_5.json') as sample:
    sample = json.load(sample)

In [99]:
print(sample)

[['city after dark.1980', {'version': 'I', 'roles': [{'role': 'Vanessa', 'gender': 'F'}, {'role': 'Nelie', 'gender': 'F'}, {'role': 'Kano', 'gender': 'F'}, {'role': 'Cora', 'gender': 'F'}, {'role': 'Bea', 'gender': 'F'}, {'role': 'Gaying', 'gender': 'F'}, {'role': 'Adelina', 'gender': 'F'}, {'role': "Vanessa's Mother", 'gender': 'F'}, {'role': 'Virgie', 'gender': 'F'}, {'role': 'Baby', 'gender': 'F'}, {'role': 'Marichu', 'gender': 'F'}, {'role': "Adelina's Client", 'gender': 'M'}, {'role': 'Manay Sharon', 'gender': 'M'}, {'role': 'Recruiter', 'gender': 'M'}, {'role': 'Mall couple', 'gender': 'M'}, {'role': 'Alex', 'gender': 'M'}, {'role': 'Pebrero', 'gender': 'M'}, {'role': 'Greggy Williams', 'gender': 'M'}, {'role': "Virgir's Husband", 'gender': 'M'}], 'year': '1980', 'title': 'City After Dark', 'id': 19412, 'num_ratings': '94', 'ratings': '7.5', 'summaries_wikipedia': [], 'venue': 'T', 'language': None, 'summaries_imdb': ["The hidden nightlife of ordinary people living in Manila unve

In [168]:
#specify stage to get the text format you need
def ie_preprocess(document, lower='false', stage="pos"):
    sentences = nltk.sent_tokenize(document)
    if stage == 'sents':
        return sentences
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    if lower == "true":
        for sent in sentences:
            for i in range(len(sent)):
                if sent[i] != sent[i].lower():
                    sent[i] = sent[i].lower()
    if stage == "words":
        return sentences
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    if stage == "pos":
        return sentences

In [219]:
# pos tag summaries and roles
for i in range(len(sample)):
    summaries_imdb = sample[i][1]["summaries_imdb"]
    summaries_wiki = sample[i][1]['summaries_wikipedia']
    roles = sample[i][1]['roles']
    if len(summaries_imdb) > 0:
        sample[i][1]["summaries_imdb_pos"] = ie_preprocess(summaries_imdb[0])
    if len(summaries_wiki) > 0:
        sample[i][1]["summaries_wikipedia_pos"] = ie_preprocess(summaries_wiki[0])
    if len(roles) > 0:
#All words in roles are originally capitalized. This confuses the tagger and all words
# are tagged NN*. So we lowercase all the words in role. 
        for j in range(len(roles)):
            roles[j]['role_pos'] = ie_preprocess(roles[j]["role"], lower="true")
#the NLP NER does not recognize named entities that are lowercase. It does do a good job
# tagging entities in sentences that are all capitalized. Therefore, we capitalize 
# all pos-tagged words in role for NER purposes.
        for i in range(len(roles[j]['role_pos'][0])):
            roles[j]['role_pos'][0][i] = list(roles[j]['role_pos'][0][i])
            roles[j]['role_pos'][0][i][0] = roles[j]['role_pos'][0][i][0].capitalize()
# tag named entities using nltk.ne_chunk
        roles[j]['role_ne'] = [nltk.ne_chunk(pts) for pts in roles[j]['role_pos']]
        if     
            

###checking the NER results
As seen below in the sample, NER does a poor job tagging named entities in the roles. This is because the roles are short phrases, not full sentences, and NER is not designed to tag short phrases.

With this finding in mind, we went on to develop other extraction methods.

In [215]:
# for i in range(len(sample[1][1]['roles'][2]['role_pos'][0])):
#     sample[1][1]['roles'][2]['role_pos'][0][i] = list(sample[1][1]['roles'][2]['role_pos'][0][i])
#     sample[1][1]['roles'][2]['role_pos'][0][i][0] = sample[1][1]['roles'][2]['role_pos'][0][i][0].lower()
#     print(sample[1][1]['roles'][2]['role_pos'][0][i][0])
    

snobby
woman


In [220]:
sample

[['city after dark.1980',
  {'id': 19412,
   'language': None,
   'num_ratings': '94',
   'ratings': '7.5',
   'roles': [{'gender': 'F',
     'role': 'Vanessa',
     'role_ne': [Tree('S', [Tree('GPE', [('Vanessa', 'NNP')])])],
     'role_pos': [[('vanessa', 'NN')]]},
    {'gender': 'F',
     'role': 'Nelie',
     'role_ne': [Tree('S', [Tree('GPE', [('Nelie', 'NNP')])])],
     'role_pos': [[('nelie', 'NN')]]},
    {'gender': 'F',
     'role': 'Kano',
     'role_ne': [Tree('S', [Tree('GPE', [('Kano', 'NNP')])])],
     'role_pos': [[('kano', 'NN')]]},
    {'gender': 'F',
     'role': 'Cora',
     'role_ne': [Tree('S', [Tree('GPE', [('Cora', 'NNP')])])],
     'role_pos': [[('cora', 'NN')]]},
    {'gender': 'F',
     'role': 'Bea',
     'role_ne': [Tree('S', [Tree('GPE', [('Bea', 'NNP')])])],
     'role_pos': [[('bea', 'NN')]]},
    {'gender': 'F',
     'role': 'Gaying',
     'role_ne': [Tree('S', [('Gaying', 'VBG')])],
     'role_pos': [[('gaying', 'VBG')]]},
    {'gender': 'F',
     'role

##Experiment with customized tagger

In [172]:
tagged = [[('As', 'A'),
     ('Constance', 'A'),
     ('becomes', 'A'),
     ('more', 'A'),
     ('and', 'CC'),
     ('more', 'JJR'),
     ('protective', 'JJ'),
     ('of', 'IN'),
     ('little', 'JJ'),
     ('Angelica', 'NNP'),
     (',', ','),
     ('a', 'DT'),
     ('ghostly', 'RB'),
     ('predator', 'NN'),
     ('begins', 'VBZ'),
     ('to', 'TO'),
     ('make', 'VB'),
     ('its', 'PRP$'),
     ('way', 'NN'),
     ('through', 'IN'),
     ('the', 'DT'),
     ('house', 'NN'),
     ('late', 'RB'),
     ('at', 'IN'),
     ('night', 'NN'),
     ('...', ':'),
     ('ANGELICA', 'NNP'),
     ('is', 'VBZ'),
     ('a', 'DT'),
     ('spellbinding', 'NN'),
     ('ghost', 'NN'),
     ('story', 'NN'),
     ('about', 'IN'),
     ('desire', 'NN'),
     (',', ','),
     ('repression', 'NN'),
     ('and', 'CC'),
     ('its', 'PRP$'),
     ('consequences', 'NNS'),
     ('...', ':')]]

In [174]:
cp = nltk.RegexpParser('CHUNK: {<A>+}')
# brown = nltk.corpus.brown
for sent in tagged:
    tree = cp.parse(sent)
    for subtree in tree.subtrees():
        if subtree.label() == 'CHUNK': print(subtree)


(CHUNK As/A Constance/A becomes/A more/A)


###experiment with NLTK Named Entity Recognition (NER)

In [195]:
# sent = nltk.corpus.treebank.tagged_sents()
sent = [[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'NNP'), ('Years', 'NNS'), ('Old', 'NNP'), (',', ','), ('Will', 'NNP'), ('Join', 'NNP'), ('The', 'NNP'), ('Board', 'NNP'), ('As', 'NNP'), ('A', 'NNP'), ('Nonexecutive', 'NNP'), ('Director', 'NNP'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'NNP'), ('chairman', 'NNP'), ('of', 'NNP'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'NNP'), ('Dutch', 'NNP'), ('publishing', 'NNP'), ('group', 'NN'), ('.', '.')], [('Rudolph', 'NNP'), ('Agnew', 'NNP'), (',', ','), ('55', 'CD'), ('years', 'NNS'), ('old', 'JJ'), ('and', 'CC'), ('former', 'JJ'), ('chairman', 'NN'), ('of', 'IN'), ('Consolidated', 'NNP'), ('Gold', 'NNP'), ('Fields', 'NNP'), ('PLC', 'NNP'), (',', ','), ('was', 'VBD'), ('named', 'VBN'), ('*-1', '-NONE-'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('of', 'IN'), ('this', 'DT'), ('British', 'JJ'), ('industrial', 'JJ'), ('conglomerate', 'NN'), ('.', '.')]]
# print(nltk.ne_chunk(sent)
pprint([nltk.ne_chunk(pts) for pts in sent])

[Tree('S', [Tree('PERSON', [('Pierre', 'NNP')]), Tree('ORGANIZATION', [('Vinken', 'NNP')]), (',', ','), ('61', 'NNP'), ('Years', 'NNS'), ('Old', 'NNP'), (',', ','), Tree('PERSON', [('Will', 'NNP'), ('Join', 'NNP'), ('The', 'NNP'), ('Board', 'NNP')]), ('As', 'NNP'), ('A', 'NNP'), ('Nonexecutive', 'NNP'), ('Director', 'NNP'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]),
 Tree('S', [Tree('PERSON', [('Mr.', 'NNP')]), Tree('PERSON', [('Vinken', 'NNP')]), ('is', 'NNP'), ('chairman', 'NNP'), ('of', 'NNP'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'NNP'), Tree('PERSON', [('Dutch', 'NNP')]), ('publishing', 'NNP'), ('group', 'NN'), ('.', '.')]),
 Tree('S', [Tree('PERSON', [('Rudolph', 'NNP')]), Tree('GPE', [('Agnew', 'NNP')]), (',', ','), ('55', 'CD'), ('years', 'NNS'), ('old', 'JJ'), ('and', 'CC'), ('former', 'JJ'), ('chairman', 'NN'), ('of', 'IN'), Tree('ORGANIZATION', [('Consolidated', 'NNP'), ('Gold', 'NNP'), ('Fields', 'NNP')]), ('PLC', 'NNP'), (',', ','), ('was', 'VBD'), (

In [182]:
type(sent)

nltk.corpus.reader.util.ConcatenatedCorpusView

The function below can potentially be useful.

In [183]:
# for n in chunked:
#     if isinstance(n, nltk.tree.Tree):               
#         if n.node == 'NP':
#             do_something_with_subtree(n)
#         else:
#             do_something_with_leaf(n)

In [184]:
print(sent[:3])

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], [('Rudolph', 'NNP'), ('Agnew', 'NNP'), (',', ','), ('55', 'CD'), ('years', 'NNS'), ('old', 'JJ'), ('and', 'CC'), ('former', 'JJ'), ('chairman', 'NN'), ('of', 'IN'), ('Consolidated', 'NNP'), ('Gold', 'NNP'), ('Fields', 'NNP'), ('PLC', 'NNP'), (',', ','), ('was', 'VBD'), ('named', 'VBN'), ('*-1', '-NONE-'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('of', 'IN'), ('this', 'DT'), ('British', 'JJ'), ('industrial', 'JJ'), ('conglomerate', 'NN'), ('.', '.')]]


###tag summaries with Stanford NER tagger

In [267]:
from nltk.tag.stanford import StanfordPOSTagger
from nltk.tag.stanford import StanfordNERTagger
#to run StanfordPostTagger and NERTagger, 
#first download these two packages from http://nlp.stanford.edu/software/CRF-NER.shtml 
#I saved the downloaded files in lib/
post = StanfordPOSTagger('lib/stanford-postagger-2014-08-27/models/english-bidirectional-distsim.tagger', 
                         'lib/stanford-postagger-2014-08-27/stanford-postagger.jar', 'utf-8') # doctest: +SKIP
# post.tag('What is the airspeed of an unladen swallow ?'.split()) # doctest: +SKIP
nert = StanfordNERTagger('lib/stanford-ner-2014-08-27/classifiers/english.all.3class.distsim.crf.ser.gz',
                         'lib/stanford-ner-2014-08-27/stanford-ner.jar', 'utf-8')



In [271]:
post.tag(sample[1][1]["summaries_imdb"])


[('In', 'IN'),
 ('Victorian', 'JJ'),
 ('England,', 'NN'),
 ('sexual', 'JJ'),
 ('repression', 'NN'),
 ('opens', 'VBZ'),
 ('a', 'DT'),
 ('rift', 'NN'),
 ('between', 'IN'),
 ('young', 'JJ'),
 ('couple', 'NN'),
 ('Constance', 'NNP'),
 ('and', 'CC'),
 ('Joseph', 'NNP'),
 ('after', 'IN'),
 ('the', 'DT'),
 ('birth', 'NN'),
 ('of', 'IN'),
 ('their', 'PRP$'),
 ('daughter', 'NN'),
 ('Angelica.', 'NNP'),
 ('As', 'IN'),
 ('Constance', 'NNP'),
 ('becomes', 'VBZ'),
 ('more', 'RBR'),
 ('and', 'CC'),
 ('more', 'RBR'),
 ('protective', 'JJ'),
 ('of', 'IN'),
 ('little', 'JJ'),
 ('Angelica,', 'NN'),
 ('a', 'DT'),
 ('ghostly', 'JJ'),
 ('predator', 'NN'),
 ('begins', 'VBZ'),
 ('to', 'TO'),
 ('make', 'VB'),
 ('its', 'PRP$'),
 ('way', 'NN'),
 ('through', 'IN'),
 ('the', 'DT'),
 ('house', 'NN'),
 ('late', 'RB'),
 ('at', 'IN'),
 ('night...', 'NNP'),
 ('ANGELICA', 'NNP'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('spellbinding', 'JJ'),
 ('ghost', 'NN'),
 ('story', 'NN'),
 ('about', 'IN'),
 ('desire,', 'NN'),
 ('repression

In [273]:
nert.tag(sample[1][1]["summaries_imdb"][0].split())
# nert.tag('Rami Eid is studying at Stony Brook University in NY'.split())

[('In', 'O'),
 ('Victorian', 'O'),
 ('England,', 'O'),
 ('sexual', 'O'),
 ('repression', 'O'),
 ('opens', 'O'),
 ('a', 'O'),
 ('rift', 'O'),
 ('between', 'O'),
 ('young', 'O'),
 ('couple', 'O'),
 ('Constance', 'PERSON'),
 ('and', 'O'),
 ('Joseph', 'PERSON'),
 ('after', 'O'),
 ('the', 'O'),
 ('birth', 'O'),
 ('of', 'O'),
 ('their', 'O'),
 ('daughter', 'O'),
 ('Angelica.', 'O'),
 ('As', 'O'),
 ('Constance', 'PERSON'),
 ('becomes', 'O'),
 ('more', 'O'),
 ('and', 'O'),
 ('more', 'O'),
 ('protective', 'O'),
 ('of', 'O'),
 ('little', 'O'),
 ('Angelica,', 'O'),
 ('a', 'O'),
 ('ghostly', 'O'),
 ('predator', 'O'),
 ('begins', 'O'),
 ('to', 'O'),
 ('make', 'O'),
 ('its', 'O'),
 ('way', 'O'),
 ('through', 'O'),
 ('the', 'O'),
 ('house', 'O'),
 ('late', 'O'),
 ('at', 'O'),
 ('night...', 'O'),
 ('ANGELICA', 'O'),
 ('is', 'O'),
 ('a', 'O'),
 ('spellbinding', 'O'),
 ('ghost', 'O'),
 ('story', 'O'),
 ('about', 'O'),
 ('desire,', 'O'),
 ('repression', 'O'),
 ('and', 'O'),
 ('its', 'O'),
 ('consequences

In [274]:
nert.tag('Rami Eid is studying at Stony Brook University in NY'.split())

[('Rami', 'PERSON'),
 ('Eid', 'PERSON'),
 ('is', 'O'),
 ('studying', 'O'),
 ('at', 'O'),
 ('Stony', 'ORGANIZATION'),
 ('Brook', 'ORGANIZATION'),
 ('University', 'ORGANIZATION'),
 ('in', 'O'),
 ('NY', 'O')]

###experiment with chunker

In [281]:
# Import the RegexpParser
from nltk.chunk import RegexpParser

# Define custom tagged entities - group NE's together 

def chunker_rules(values):
    # Define  custom grammar (modified to be a valid regex).
    grammar = r'''
        PERSON:
                {<PERSON><O><PERSON>+}
                {<PERSON>+}
            '''
    cp = nltk.RegexpParser(grammar) # Create an instance of your custom parser.
    return cp.parse(values)         # Parse!

def entity_chunker(tagged_docs):
    chunks = []
#     for doc in tagged_docs:
    tree = chunker_rules(tagged_docs)
    for subtree in tree.subtrees():
#             if (subtree.node == 'WIDOW'):
        leaflist = [leaf[0] for leaf in subtree.leaves()]
        chunks.append(' '.join(leaflist))
    return chunks
 


In [282]:
example = [('Rami', 'PERSON'),
 ('Eid', 'PERSON'),
 ('is', 'O'),
 ('studying', 'O'),
 ('at', 'O'),
 ('Stony', 'ORGANIZATION'),
 ('Brook', 'ORGANIZATION'),
 ('University', 'ORGANIZATION'),
 ('in', 'O'),
 ('NY', 'O')]

In [283]:
entity_chunker(example)

['Rami Eid is studying at Stony Brook University in NY', 'Rami Eid']

In [301]:
###can chunker define alternative grammers??"
cp = nltk.RegexpParser('PERSON:{<PERSON>+}')
# brown = nltk.corpus.brown
# for sent in tagged:
tree = cp.parse(example)
for subtree in tree.subtrees():
    if subtree.label() == 'PERSON': print(subtree)

(PERSON Rami/PERSON Eid/PERSON)


In [300]:
print(tree)

(S
  (PERSON Rami/PERSON Eid/PERSON)
  is/O
  studying/O
  at/O
  Stony/ORGANIZATION
  Brook/ORGANIZATION
  University/ORGANIZATION
  in/O
  NY/O)


###Replace more /O with names from the SSN database.